# Evaluation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import regex as re

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pwd

/content


In [38]:
# Nhập dữ liệu
with open('/content/drive/MyDrive/ViNLU/dev-results/seq.out', 'r', encoding= 'utf-8') as file:
    out_lines = file.readlines()
    file.close()

with open('/content/drive/MyDrive/ViNLU/dev-results/seq.in', 'r', encoding= 'utf-8') as file:
    in_lines = file.readlines()
    file.close()

with open('/content/drive/MyDrive/ViNLU/dev-results/label', 'r', encoding= 'utf-8') as file:
    out_labels = file.readlines()
    file.close()

with open('/content/drive/MyDrive/ViNLU/dev-results/predict.out', 'r', encoding= 'utf-8') as file:
    test_lines = file.readlines()
    file.close()

In [27]:
# Số samples trong bộ test
len(out_lines), len(test_lines)

(392, 392)

In [40]:
# Chuẩn hóa
out_lines = [x.strip() for x in out_lines]
in_lines = [x.strip() for x in in_lines]
out_labels = [x.strip() for x in out_labels]
test_lines = [x.strip() for x in test_lines]

In [29]:
out_lines[0]

'O O O O O B-devicedevice I-devicedevice'

In [30]:
test_lines[0]

'<smart.home.check.status> -> anh ơi thiết_bị là đầy_đủ [rgb:B-devicedevice] [4:I-devicedevice]'

In [31]:
# Chuyển định dạng
def test2out(test_line):
    words = test_line.split()
    intent = words[0]
    words = words[2:]
    str = []
    for word in words:
        if word[0] != '[':
            str.append('O')
        else:
            info = word[1:-1]
            str.append(info.split(':')[-1])
    slots = ' '.join(str)
    return intent[1:-1], slots

## Khảo sát intent

In [32]:
# Kiểm tra
intent, slots = test2out(test_lines[0])
intent, out_labels[0]

('smart.home.check.status', 'smart.home.decrease.percentage')

In [35]:
diff_ids = []
for i in range(len(test_lines)):
  label, gt_label = test2out(test_lines[i])[0], out_labels[i]
  if label != gt_label:
    diff_ids.append(i)

print(len(diff_ids))

61


In [37]:
# Các lỗi gặp phải
diffs = []
for i in diff_ids:
  label, gt_label = test2out(test_lines[i])[0], out_labels[i]
  diffs.append(f'{label}||{gt_label}')

diffs = {i:diffs.count(i) for i in set(diffs)}
diffs = dict(sorted(diffs.items(), key=lambda item: -item[1]))
diffs # test||out

{'smart.home.decrease.level||smart.home.decrease.percentage': 12,
 'smart.home.set.level||smart.home.set.percentage': 11,
 'smart.home.check.status||smart.home.decrease.percentage': 10,
 'smart.home.device.onoff||smart.home.set.level': 6,
 'smart.home.set.level||smart.home.check.status': 5,
 'smart.home.check.status||smart.home.increase.percentage': 5,
 'smart.home.increase.level||smart.home.decrease.level': 5,
 'smart.home.device.onoff||smart.home.set.percentage': 4,
 'smart.home.increase.percentage||smart.home.device.onoff': 2,
 'smart.home.set.percentage||smart.home.device.onoff': 1}

In [44]:
# Các câu chứa lỗi intent
[in_lines[i] for i in diff_ids]

['anh ơi thiết_bị là đầy_đủ rgb 4',
 'chào giảm giúp tôi đèn hoa trong phòng con',
 'tôi muốn giảm độ sáng của đèn ngủ 2',
 'tôi muốn giảm độ sáng của bóng hoa 2',
 'chào bạn kiểm_soát được điện ở vườn sau 2 trong nhà không',
 'tôi muốn giảm độ sáng của kịch_bản 3',
 'chào bạn kiểm_soát được bóng compact thứ 4 ở phòng ông_bà 4 trong nhà không',
 'chào bạn kiểm_soát được đèn sân 3 ở phòng ăn 3 trong nhà không',
 'giúp tui giảm thiết_bị đèn bàn thứ 3',
 'giúp tui giảm thiết_bị rèm ngang',
 'anh ơi thiết_bị là đầy_đủ màn cửa đơn',
 'chào bạn kiểm_soát được bóng downlight 2 ở phòng ngủ con trai 5 trong nhà không',
 'anh ơi thiết_bị là đầy_đủ quạt_treo_tường 3',
 'giúp tui giảm thiết_bị downlight 1',
 'giúp tui giảm thiết_bị cột đèn 3',
 'chào giảm giúp tôi màn cửa đơn 4 trong tủ giữ đồ 3',
 'chào bạn kiểm_soát được quạt hút mùi thứ 1 ở garage trong nhà không',
 'giúp tui giảm thiết_bị máy_lạnh thứ 3',
 'anh ơi thiết_bị là đầy_đủ đèn tranh 1',
 'anh ơi thiết_bị là đầy_đủ bóng sân 3',
 'tôi 

### Khảo sát intent logits

In [48]:
# Load logits
intent_logits = np.loadtxt('/content/drive/MyDrive/ViNLU/dev-results/intent_logits.txt')
intent_logits.shape

(392, 10)

In [57]:
# Confidence
confidences = np.max(intent_logits, axis= 1)
np.array(sorted(confidences)[:45]) # Đặt threshold = 0.98

array([0.51555061, 0.60156488, 0.67853969, 0.68383741, 0.68993062,
       0.69622463, 0.70404553, 0.75817531, 0.76219338, 0.76570511,
       0.77415633, 0.79214472, 0.8672868 , 0.87057364, 0.88746619,
       0.9003073 , 0.9052797 , 0.91257495, 0.92285335, 0.94513863,
       0.95795619, 0.9581756 , 0.9675647 , 0.97845459, 0.98006368,
       0.98016351, 0.98197168, 0.98262089, 0.98355228, 0.98486996,
       0.98922276, 0.98977029, 0.99045813, 0.99081087, 0.99288952,
       0.99324495, 0.99389333, 0.9940573 , 0.99452221, 0.99484926,
       0.9954052 , 0.99581665, 0.99582219, 0.9958747 , 0.9964065 ])

In [68]:
hard_ids = []
for i in range(intent_logits.shape[0]):
  if i in diff_ids:
    continue
  if confidences[i] < 0.98:
    hard_ids.append(i)

hard_ids = sorted(hard_ids, key= lambda x:confidences[x])
for id in hard_ids:
  print(id, confidences[id], out_labels[id])

310 0.5155506134033203 smart.home.set.level
30 0.6785396933555603 smart.home.decrease.percentage
341 0.6838374137878418 smart.home.device.onoff
42 0.6899306178092957 smart.home.decrease.percentage
13 0.7040455341339111 smart.home.decrease.percentage
178 0.8705736398696899 smart.home.set.percentage
381 0.900307297706604 smart.home.device.onoff
173 0.9125749468803406 smart.home.set.percentage
321 0.9228533506393433 smart.home.set.level
179 0.9451386332511902 smart.home.set.percentage
370 0.9579561948776245 smart.home.device.onoff


In [62]:
# Các câu model dự đoán ĐÚNG nhưng intent confidence THẤP. Các câu được sắp xếp theo thứ tự confidences tăng dần
[in_lines[i] for i in hard_ids]

['bạn đặt hộ mình đèn cổng lên với',
 'chào giảm giúp tôi bòng hắt trần thứ 4 trong phòng nghỉ 4',
 'bật rèm cửa đi cái',
 'chào giảm giúp tôi đèn nhiều màu 1 trong phòng ông chủ 5',
 'chào giảm giúp tôi đèn trong phòng nữ',
 'mình nhờ bạn mở thiết_bị bóng âm trần 1 ở cửa_hàng nhé',
 'giúp tôi bật cái đèn sắc_màu nhé',
 'hãy đóng đèn cổng 2 trong phòng',
 'bạn cài_đặt hộ mình đèn ốp tường lên với',
 'hãy đóng bóng thờ thứ 3 trong nhà_tắm 1',
 'giúp tôi bật cái bóng trụ cổng 1 nhé']

## Khảo sát slot filling

In [63]:
# Kiểm tra
intent, slots = test2out(test_lines[0])
slots

'O O O O O B-devicedevice I-devicedevice'

In [64]:
out_lines[0]

'O O O O O B-devicedevice I-devicedevice'

In [76]:
# Ghi file
outpath = 'slot_diff.txt'
lines = ['id||seq_test||seq_out\n']
raw_lines = ['id||output||ground_truth\n']

for i in range(len(out_lines)):
    intent, slots = test2out(test_lines[i])
    if slots != out_lines[i]:
        lines.append('||'.join([str(i), slots, out_lines[i]]) + '\n')
        raw_lines.append('||'.join([str(i), slots, out_lines[i]]))

with open(outpath, 'w', encoding= 'utf-8') as file:
    file.writelines(lines)
    file.close()

raw_lines

['id||output||ground_truth\n',
 '6||O O O O B-devicedevice O B-roomroom I-roomroom I-roomroom O B-roomroom O||O O O O B-devicedevice O B-roomroom I-roomroom I-roomroom O O O',
 '8||O O O O B-devicedevice I-devicedevice I-devicedevice I-devicedevice O B-roomroom I-roomroom I-roomroom O B-roomroom O||O O O O B-devicedevice I-devicedevice I-devicedevice I-devicedevice O B-roomroom I-roomroom I-roomroom O O O',
 '10||O O O O B-devicedevice I-devicedevice I-devicedevice O B-roomroom I-roomroom I-roomroom O B-roomroom O||O O O O B-devicedevice I-devicedevice I-devicedevice O B-roomroom I-roomroom I-roomroom O O O',
 '15||O B-commandcommand I-commandcommand O B-devicedevice I-devicedevice I-devicedevice B-final-valuesyspercentage I-final-valuesyspercentage O O||O B-commandcommand I-commandcommand O B-devicedevice I-devicedevice I-devicedevice B-change-valuesyspercentage I-change-valuesyspercentage O O',
 '20||O O O O B-devicedevice I-devicedevice I-devicedevice O B-roomroom I-roomroom I-roomr

In [81]:
# Các lỗi gặp phải
diffs = []
slot_diff_ids = []
for t in range(len(lines)):
    words = [x.strip() for x in lines[t].split('||')]
    if words[0] != 'id':
      id = int(words[0])
      slot_diff_ids.append(id)

    test = words[1].split()
    out = words[2].split()

    for i in range(len(test)):
        if test[i] != out[i]:
            diffs.append(f'{test[i]}||{out[i]}')

diffs = {i:diffs.count(i) for i in set(diffs)}
diffs = dict(sorted(diffs.items(), key=lambda item: -item[1]))
diffs # test||out

{'B-commandcommand||O': 31,
 'B-roomroom||O': 8,
 'O||B-commandcommand': 6,
 'B-sysnumbersysnumber||O': 5,
 'I-roomroom||B-floornumberfloornumber': 3,
 'O||B-sysnumbersysnumber': 3,
 'I-roomroom||B-roomroom': 2,
 'B-roomroom||B-floornumberfloornumber': 2,
 'B-final-valuesyspercentage||B-change-valuesyspercentage': 2,
 'B-colorcolor||O': 2,
 'I-final-valuesyspercentage||I-change-valuesyspercentage': 2,
 'I-colorcolor||B-colorcolor': 2,
 'I-roomroom||I-floornumberfloornumber': 2,
 'I-roomroom||B-commandcommand': 1,
 'I-devicedevice||B-sysnumbersysnumber': 1,
 'B-final-valuesysnumber||B-sysnumbersysnumber': 1,
 'I-statusstatus||B-commandcommand': 1,
 'O||B-colorcolor': 1,
 'seq_test||seq_out': 1,
 'O||I-commandcommand': 1,
 'B-sysnumbersysnumber||I-devicedevice': 1,
 'B-colorcolor||I-colorcolor': 1,
 'I-commandcommand||O': 1,
 'B-sysnumbersysnumber||B-final-valuesysnumber': 1}

In [83]:
# Các câu bị sai slot
[in_lines[i] for i in slot_diff_ids]

['chào bạn kiểm_soát được điện ở vườn sau 2 trong nhà không',
 'chào bạn kiểm_soát được bóng compact thứ 4 ở phòng ông_bà 4 trong nhà không',
 'chào bạn kiểm_soát được đèn sân 3 ở phòng ăn 3 trong nhà không',
 'làm_ơn hạ thấp đỡ bóng đứng 2 7 phần_trăm giúp tôi',
 'chào bạn kiểm_soát được bóng downlight 2 ở phòng ngủ con trai 5 trong nhà không',
 'làm_ơn hạ xuống đỡ đèn bếp 3 phần_trăm giúp tôi',
 'làm_ơn giảm đỡ đèn rọi 16 phần_trăm giúp tôi',
 'chào bạn kiểm_soát được quạt hút mùi thứ 1 ở garage trong nhà không',
 'bạn chỉnh giúp kiểm_tra màn cuốn thứ 1 ở phòng họp gia_đình xem bật không nhé',
 'bạn chỉnh giúp kiểm_tra đèn sân thứ 4 ở hè sau 5 xem bật không nhé',
 'bóng hắt tường phòng ngủ các con 4 còn bật cập_nhật',
 'bạn chỉnh giúp kiểm_tra loa còi ở phòng sách 1 xem bật không nhé',
 'bóng 2 phòng ngủ 5 đang tắt kiểm_tra',
 'bạn chỉnh giúp kiểm_tra bóng trang_trí thứ 2 ở quầy lễ_tân xem bật không nhé',
 'bóng vách 1 nhà giữ đồ còn tắt đúng không',
 'bạn chỉnh giúp kiểm_tra đèn nhi